In [12]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
open-webui 0.3.35 requires aiohttp==3.10.8, but you have aiohttp 3.11.11 which is incompatible.
open-webui 0.3.35 requires boto3==1.35.0, but you have boto3 1.34.69 which is incompatible.
open-webui 0.3.35 requires chromadb==0.5.9, but you have chromadb 0.5.20 which is incompatible.
open-webui 0.3.35 requires google-generativeai==0.7.2, but you have google-generativeai 0.5.4 which is incompatible.
open-webui 0.3.35 requires langchain==0.2.15, but you have langchain 0.3.13 which is incompatible.
open-webui 0.3.35 requires langchain-community==0.2.12, but you have langchain-community 0.3.13 which is incompatible.
open-webui 0.3.35 requires pypdf==4.3.1, but you have pypdf 5.1.0 which is incompatible.


In [1]:
%pip install langchain llama-index


Note: you may need to restart the kernel to use updated packages.


In [13]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

In [7]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

assert len(docs) == 1
print(f"Total characters: {len(docs[0].page_content)}")

USER_AGENT environment variable not set, consider setting it to identify your requests.


Total characters: 43130


In [8]:
print(docs[0].page_content[:500])





      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 66 sub-documents.


In [16]:
%pip install -qU langchain-core



Note: you may need to restart the kernel to use updated packages.


In [19]:
import os
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

# Configure Ollama LLM
ollama_llm = Ollama(
    #model="llama3.2:latest",
    model="mistral:7b",
    base_url="http://localhost:11434",
    temperature=0.1
)

# Configure embedding model
ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text:latest",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0}
)

Settings.llm = ollama_llm
Settings.embed_model = ollama_embedding

In [36]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.base import Embeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms.base import BaseLLM
import requests
import json


# Step 1: Custom LLM Wrapper for Ollama
class OllamaLLM(BaseLLM):
    def __init__(self, model_name: str, base_url: str = "http://localhost:11434", temperature: float = 0.1):
        self.model_name = model_name
        self.base_url = base_url
        self.temperature = temperature

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        headers = {"Content-Type": "application/json"}
        data = {
            "model": self.model_name,
            "prompt": prompt,
            "temperature": self.temperature,
        }
        if stop:
            data["stop"] = stop
        response = requests.post(f"{self.base_url}/api/completion", headers=headers, data=json.dumps(data))
        if response.status_code == 200:
            return response.json()["choices"][0]["text"]
        else:
            raise Exception(f"Ollama API error: {response.text}")

    def _generate(self, prompts: List[str], stop: Optional[List[str]] = None) -> List[str]:
        # Generate responses for multiple prompts
        return [self._call(prompt, stop=stop) for prompt in prompts]

    @property
    def _llm_type(self) -> str:
        return "ollama"

# Step 2: Custom Embedding Class for Ollama
class OllamaEmbedding(Embeddings):
    def __init__(self, model_name: str, base_url: str = "http://localhost:11434"):
        self.model_name = model_name
        self.base_url = base_url

    def embed_documents(self, texts):
        return [self._embed(text) for text in texts]

    def embed_query(self, text):
        return self._embed(text)

    def _embed(self, text):
        headers = {"Content-Type": "application/json"}
        data = {
            "model": self.model_name,
            "prompt": text
        }
        response = requests.post(f"{self.base_url}/api/embed", headers=headers, data=json.dumps(data))
        if response.status_code == 200:
            return response.json()["embedding"]
        else:
            raise Exception(f"Ollama API error: {response.text}")

NameError: name 'Optional' is not defined

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(ollama_embedding)

In [27]:
document_ids = vector_store.add_documents(documents=all_splits)

print(document_ids[:3])

AttributeError: 'OllamaEmbedding' object has no attribute 'embed_documents'

In [33]:
%pip install chromadb



  Using cached opentelemetry_instrumentation-0.48b0-py3-none-any.whl.metadata (6.1 kB)
  Using cached opentelemetry_semantic_conventions-0.48b0-py3-none-any.whl.metadata (2.4 kB)
  Using cached opentelemetry_util_http-0.48b0-py3-none-any.whl.metadata (2.5 kB)
  Using cached opentelemetry_api-1.27.0-py3-none-any.whl.metadata (1.4 kB)
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
  Using cached opentelemetry_instrumentation_asgi-0.48b0-py3-none-any.whl.metadata (2.0 kB)
  Attempting uninstall: opentelemetry-instrumentation-asgi
    Found existing installation: opentelemetry-instrumentation-asgi 0.48b0
    Uninstalling opentelemetry-instrumentation-asgi-0.48b0:
      Successfully uninstalled opentelemetry-instrumentation-asgi-0.48b0
  Attempting uninstall: opentelemetry-instrumentation-fastapi
    Found existing installation: opentelemetry-instrumentation-fastapi 0.48b0
    Uninstal

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-vector-stores-chroma 0.3.0 requires llama-index-core<0.12.0,>=0.11.0, but you have llama-index-core 0.12.6 which is incompatible.
open-webui 0.3.35 requires aiohttp==3.10.8, but you have aiohttp 3.11.11 which is incompatible.
open-webui 0.3.35 requires boto3==1.35.0, but you have boto3 1.34.69 which is incompatible.
open-webui 0.3.35 requires chromadb==0.5.9, but you have chromadb 0.5.20 which is incompatible.
open-webui 0.3.35 requires google-generativeai==0.7.2, but you have google-generativeai 0.5.4 which is incompatible.
open-webui 0.3.35 requires langchain==0.2.15, but you have langchain 0.3.13 which is incompatible.
open-webui 0.3.35 requires langchain-community==0.2.12, but you have langchain-community 0.3.13 which is incompatible.
open-webui 0.3.35 requires pypdf==4.3.1, but you have pypdf 5.1.

In [35]:
from sentence_transformers import SentenceTransformer
from langchain.embeddings.base import Embeddings

# Step 1: Custom Embedding Class Using SentenceTransformer
class SentenceTransformerEmbedding(Embeddings):
    def __init__(self, model_name="all-MiniLM-L6-v2"):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts):
        return self.model.encode(texts, convert_to_tensor=True).tolist()

    def embed_query(self, text):
        return self.model.encode(text, convert_to_tensor=True).tolist()

from langchain.vectorstores import Chroma

def create_vectorstore(documents, embedding_model):
    vectorstore = Chroma.from_documents(documents, embedding_model)
    return vectorstore

# Step 3: Main Program
if __name__ == "__main__":
    # Path to the text file
    file_path = "../0-Data/paul_graham_short.txt"  # Replace with the actual file path
    
    # Step 1: Load the text file
    print("Loading documents...")
    documents = load_documents(file_path)
    
    # Step 2: Create embeddings and a vectorstore
    print("Creating vectorstore...")
    embedding_model = SentenceTransformerEmbedding()
    vectorstore = create_vectorstore(documents, embedding_model)
    
    # Step 3: Set up the QA chain with Ollama
    print("Setting up QA chain...")
    ollama_llm = OllamaLLM(model_name="llama2:latest", base_url="http://localhost:11434", temperature=0.1)
    qa_chain = setup_qa_chain(vectorstore, ollama_llm)
    
    # Step 4: Query the chain
    query = "What does Paul Graham say about startups?"
    print("Querying the system...")
    response = qa_chain.run(query)
    
    # Print the response
    print("Query Response:")
    print(response)


Loading documents...
Creating vectorstore...
Setting up QA chain...


TypeError: Can't instantiate abstract class OllamaLLM with abstract methods _generate, _llm_type